In [1]:
%load_ext autoreload
%autoreload 2

import os,sys
os.environ['BASE_DIR'] =  '/Users/macico/Dropbox/btc'
os.environ['COMMON_DIR'] = os.path.join(os.environ['BASE_DIR'], "geco_commons")
sys.path.append(os.path.join(os.environ['COMMON_DIR'],"util" ))
from setting import *

In [3]:


import master
from master import *
from datetime import datetime as dt

In [4]:
# args=['--execute_mode',"prepro",'--config_source','ini','--model_id','test001',
#       '-train_sd','20210101','-train_ed','20210103',
#       '-valid_sd','20210104','-train_ed','20210105',
#       '-test_sd','20210106','-test_ed','20210107']
args=['--symbol','BTC', '--execute_mode',"gtrain",
      '--model_id','test002','--model_name','slstm',
      '--config_source','ini','--config_mode','dev',
      '--user',"Misaka00001", '--source','ipynb',
      '-train_sd','20210101','-train_ed','20210103',
      '-valid_sd','20210104','-valid_ed','20210105',
      '-test_sd','20210106','-test_ed','20210107']

In [14]:
parser = make_parser()
arg_dict = vars(parser.parse_args(args))

#meta
_id = arg_dict["model_id"]
config_mode = arg_dict["config_mode"].upper()
model_name = arg_dict["model_name"].upper()

# period
train_start = arg_dict["train_start_date"] if "train_start_date" in arg_dict.keys() else None
train_end = arg_dict["train_end_date"] if "train_end_date" in arg_dict.keys() else None
valid_start = arg_dict["valid_start_date"] if "valid_start_date" in arg_dict.keys() else None
valid_end = arg_dict["valid_end_date"]if "valid_end_date" in arg_dict.keys() else None
test_start = arg_dict["test_start_date"] if "test_start_date" in arg_dict.keys() else None
test_end = arg_dict["test_end_date"] if "test_end_date" in arg_dict.keys() else None

# sym
sym = arg_dict["symbol"] 

#mlflow
mlflow_tags = {
    MLFLOW_USER:arg_dict["user"],
    MLFLOW_SOURCE_NAME:arg_dict["source"],
    MLFLOW_RUN_NAME: f"TRAIN_{dt.now().strftime('%y%m%d%H%M%s')}"
}

# load meta info
om = OperateMaster()
om.load_meta(_id, model_name,  config_mode )
om.set_mlflow_tags(mlflow_tags)

# load conofigs into each module
om.init_prepro()
om.init_learning()

if arg_dict["execute_mode"] == "prepro":
    om.init_dataGen()
    om.preprocessing(sym, train_start, train_end, valid_start, valid_end, test_start, test_end)
elif arg_dict["execute_mode"] == "train":
    om.train()
elif arg_dict["execute_mode"] == "gtrain":
    om.gtrain()
elif arg_dict["execute_mode"] == "rpredict":
    om.init_dataGen(remote=True)
    om.predict()

else:
    pass

<ModelVersion: creation_timestamp=1639566838516, current_stage='None', description='', last_updated_timestamp=1639566838516, name='test002_test', run_id='92a358b8066943c8bb646739a5e313ae', run_link='', source='/Users/macico/Dropbox/btc/mlruns/2/92a358b8066943c8bb646739a5e313ae/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>

In [60]:
om.le.mlwriter.client.get_run(best_run[0].info.run_id)

<Run: data=<RunData: metrics={'test_acc': 0.6089201877934273,
 'train_loss': 0.7199654926115008,
 'valid_acc': 0.5555555555555556,
 'valid_loss': 0.727906052271525}, params={'batch_size': '64',
 'loss_fn': 'BCELogitLoss',
 'n_epochs': '15',
 'optimizer': 'adam'}, tags={'mlflow.runName': 'TRAIN_21121520411639568475_0_1_2_3_4_5_6_7_8_9_10_11',
 'mlflow.source.name': 'ipynb',
 'mlflow.user': 'Misaka00001'}>, info=<RunInfo: artifact_uri='/Users/macico/Dropbox/btc/mlruns/2/6f3cf06067a54db289b503f16a9086df/artifacts', end_time=1639568589192, experiment_id='2', lifecycle_stage='active', run_id='6f3cf06067a54db289b503f16a9086df', run_uuid='6f3cf06067a54db289b503f16a9086df', start_time=1639568576581, status='FINISHED', user_id='Misaka00001'>>

In [61]:

om.le.mlwriter.create_model_version("test002_test",run_id=best_run[0].info.run_id)

In [45]:
om.le.mlwriter.client.get_experiment_by_name("test002").experiment_id

'2'

In [46]:
om.le.mlwriter.register_model("test002_test")

/Users/macico/Dropbox/btc/aleister/src/mlflow_writer.py:41: DeprecationWarning:

The 'warn' method is deprecated, use 'warning' instead



In [48]:
mv = om.le.mlwriter.create_model_version("test002_test")

In [50]:
mv.info

AttributeError: 'NoneType' object has no attribute 'info'

In [42]:
import mlflow
from mlflow.entities import ViewType
#serch runs
# query = "params.model = 'CNN' and params.layers = '10' and metrics.`prediction accuracy` >= 0.945"
query = None
run_info = om.le.mlwriter.client.search_runs(experiment_ids=["1"], filter_string=query, run_view_type=ViewType.ACTIVE_ONLY)[0]
uri =  os.path.join(run_info.info.artifact_uri, "model" ) 
mlflow.pytorch.load_model(uri)

SimpleLSTM(
  (lstm): LSTM(7, 32, num_layers=2, batch_first=True)
  (fc): Linear(in_features=32, out_features=3, bias=True)
)

In [43]:
model_info = le.mlwriter.client.get_latest_versions("test_model")[0]
mlflow.pytorch.load_model(model_info.source)

NameError: name 'le' is not defined

In [ ]:
for  i in le.ys:
    print(i)

In [ ]:
import joblib
load_path="/Users/macico/Dropbox/btc_data/model/test001/X_test.jbl"
with open(load_path, mode="rb") as f:
    data = joblib.load(f)
data